In [ ]:
#importing the libraries needed 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
#Declaring the headers 
headers = {"Accept-Language": "en-US,en;q=0.5"}
movie_title = []
year = []
rating=[]
director =[]
cast = []
synop = []
genre=[]
movie_duration=[]
pages = np.arange(2)
basic_link="/search/title/?release_date=1980-01-01,2019-10-31&after=WzM2NjE2OCwidHQ2OTczODU4IiwyMjU2NzBd"
for page in pages:
    add_end="&ref_=adv_nxt"
    add_st="https://www.imdb.com"
    full_link=add_st+basic_link+add_end
    page = requests.get(full_link)
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,5))
    for info in movie_data:
        name= info.find('div',class_="lister-item-content").find('h3',class_="lister-item-header").find('a').text
        #name = info.h3.a.text
        movie_title.append(name)
        
        # year of release
        release = info.find('h3',class_="lister-item-header").find('span',class_="lister-item-year text-muted unbold").text
        ab=release.split("–")
        
        if(len(ab)==2):
            if(ab[1]!=" )"):    
                bn=ab[0]+str("-")+ab[1]
                year.append(bn)
            else:
                year.append(ab[0])
        elif(len(ab)==1):
            year.append(ab[0])   

        try:
            rate= info.find('div',class_="ratings-bar").find('div',class_="inline-block ratings-imdb-rating").find('strong').text
            rating.append(rate)
        except:
            rating.append(0)
        
        # Description of the Movies
        try:
            describe = info.findAll('p', class_ = 'text-muted')
            description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*'
            if(description_=="Add a Plot"):
                synop.append("N/A")
            else:
                synop.append(description_)
        except:
            print("no description")
            synop.append("N/A")
        try:
            
            direct=info.find('div',class_="lister-item-content").find_all('p')
            dr=direct[2].find_all('a') if direct[2].find_all('a') else director.append("N/A")
            dr=direct[2].find_all('a') if direct[2].find_all('a') else cast.append("N/A")
            #print(dr)
            dire=dr[0].text
            director.append(dire)
            cst=[]
            if(len(dr)>1):
                for i in range(1,len(dr)):
                    cst.append(dr[i].text)
                cast.append(cst)           
            else:
                cast.append("N/A")
            gen=info.find('div',class_="lister-item-content").find('span',class_="genre").text.replace('\n','').replace('            ','')    
            #gen=info.find('div',class_="lister-item-content").find('span',class_="genre").text.replace('\n','')
            # duration of the movie
            tim = info.p.find('span', class_ = "runtime").text if info.p.find('span', class_ = "runtime").text else "N/A"
            genre.append(gen)
           # print(tim)
            movie_duration.append(tim)

        except:
           # director.append("N/A ")
           # cast.append("N/A")
            genre.append("N/A")
            movie_duration.append("N/A")
            
    next_pagelink=soup.find('div', attrs = {'class': 'desc'})
    nextt=next_pagelink.find('a',class_="lister-page-next next-page")
    kk=nextt['href']
    #print(kk)
    basic_link=""
    basic_link=kk
movie_list = pd.DataFrame({ "Movie Name": movie_title,"Duration":movie_duration,"Year of Release" : year, "Gnere": genre,"Movie Rating": rating, "Direcror": director, "Cast" : cast, "Description": synop})
movie_list.to_csv("scrapData7.csv")